In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import random

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 # image processing library
from PIL import Image # another image library
import glob # finding files
from tqdm.notebook import tqdm # progress bars

# Pytorch Machine Learning Library
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, random_split, Dataset
from torchvision import transforms

import gc

TRAIN_PATH = "../SIIM Dataset/224px/train/train/"
TEST_PATH = "../SIIM Dataset/224px/test/test/"
CSV_PATH = "../SIIM Dataset/train.csv"

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        
set_seed(42)

## Data Preprocessing

In [2]:
# Dataset Class for loading the COVID Data with PyTorch functionality

labels = ['Negative for Pneumonia', 'Indeterminate Appearance', 'Typical Appearance', 'Atypical Appearance']

class SIIM_Dataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
            
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        
        # Getting the image
        self.data.head()
        img_id, study_id = self.data[['ImageInstanceUID', 'StudyInstanceUID']].iloc[idx]
        filepath = glob.glob(self.root_dir + study_id + "*" + img_id + ".jpg")[0]
        img = cv2.imread(filepath)
        img = Image.fromarray(img)

        #Getting the label
        label_str = self.data['label_id'].iloc[idx]
        label = labels.index(label_str)
        
        if self.transform:
            img = self.transform(img)
    
        return img, label
        

In [3]:
# Creating the data loaders that will be used during training

test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize([0.44099547, 0.51991787, 0.60964888], [0.22431996, 0.20356079, 0.26086323])
                                    ])

train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                     transforms.RandomResizedCrop((32, 32),scale=(0.8,1.0),ratio=(0.9,1.1)),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.44099547, 0.51991787, 0.60964888], [0.22431996, 0.20356079, 0.26086323])
                                    ])
val_dataset = SIIM_Dataset(CSV_PATH, TRAIN_PATH, transform=train_transform)
train_dataset = SIIM_Dataset(CSV_PATH, TRAIN_PATH, transform=test_transform)

set_seed(42)
train_set, _ = random_split(train_dataset, [5334, 1000])
set_seed(42)
_, val_set = random_split(val_dataset, [5334, 1000])

train_loader = DataLoader(train_set, batch_size=128, shuffle=True, drop_last=False, pin_memory=True, num_workers=4)
val_loader = DataLoader(val_set, batch_size=128, shuffle=False, drop_last=False, num_workers=4)

## ResNet Implementation

In [4]:
#Activation Functions
act_fn_by_name = {
    "tanh": nn.Tanh,
    "relu": nn.ReLU,
    "leakyrelu": nn.LeakyReLU,
    "gelu": nn.GELU
}

In [5]:
#Individual Blocks within the ResNet Architecture
class ResNetBlock(nn.Module):
    def __init__(self, c_in, act_fn, c_out=-1, downsample=False):
        super().__init__()
        if not downsample:
            c_out = c_in
        
        #Neural Network representing F
        self.net = nn.Sequential(
            nn.Conv2d(c_in, c_out, kernel_size=3, padding=1, stride=1 if not downsample else 2, bias=False),
            nn.BatchNorm2d(c_out),
            act_fn(),
            nn.Conv2d(c_out, c_out, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(c_out)
        )
        
        #Downsampling network for the input in order to facilitate F(x) + x
        self.downsample = nn.Conv2d(c_in, c_out, kernel_size=1, stride=2) if downsample else None
        self.act_fn = act_fn()
        
    def forward(self, x):
        z = self.net(x)
        if self.downsample is not None:
            x = self.downsample(x)
        out = z + x
        out = self.act_fn(out)
        return out
        

In [6]:
#ResNet Model Implementation
class ResNet(nn.Module):
    def __init__(self, num_classes, num_blocks=[3, 3, 3, 3, 3, ], c_hidden=[64, 128, 256, 1024, 2048], act_fn_name='relu'):
        super().__init__()
        self.num_blocks = num_blocks
        self.c_hidden = c_hidden
        self.num_classes = num_classes
        self.act_fn = act_fn_by_name[act_fn_name]
        self.act_fn_name = act_fn_name
        self._create_network()
        self._init_params()
    
    def _create_network(self):
        
        self.input_net = nn.Sequential(
            nn.Conv2d(3, self.c_hidden[0], kernel_size=7, padding=3, stride=2, bias=False),
            nn.BatchNorm2d(self.c_hidden[0]),
            nn.MaxPool2d(kernel_size=3, padding=1, stride=2),
            self.act_fn()
        )
        
        blocks = []
        for block_idx, block_count in enumerate(self.num_blocks):
            for bc in range(block_count):
                downsample = (bc == 0 and block_idx > 0)
                blocks.append(
                    ResNetBlock(c_in=self.c_hidden[block_idx if not downsample else (block_idx - 1)],
                                act_fn=self.act_fn,
                                c_out=self.c_hidden[block_idx],
                                downsample=downsample))
        self.blocks = nn.Sequential(*blocks)
        
        self.output_net = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(self.c_hidden[-1], self.num_classes)
        )
    
    def _init_params(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity=self.act_fn_name)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.input_net(x)
        x = self.blocks(x)
        x = self.output_net(x)
        return x
            
                


In [7]:
# Finding a Device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Device cuda


In [8]:
def train_model(model, nb_epochs, lr=0.01):
    model.to(device)
    loss_module = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    epoch_bar = tqdm(range(nb_epochs), desc="Epoch Progress")
    train_bar = tqdm(train_loader, desc="Training Batches")
    val_bar = tqdm(val_loader, desc="Validating Batches")
    for epoch in epoch_bar:
        #Training the model on the given batch
        train_losses = []
        for batch in train_loader:
            train_bar.update()
            x, y = batch
            y = y.long()
            x, y = x.to(device), y.to(device)
            #Calculating forward pass of the model
            output = model(x)
            
            #Calculating loss
            loss = loss_module(output, y)
            train_losses.append(loss.item())
            #Cleaning gradients
            optimizer.zero_grad()
            
            #Backwward propagation of partial gradients
            loss.backward()
            optimizer.step()
            
            del x
            del y
            gc.collect()
        
        train_bar.refresh()
        #Same steps, but calculating the validation loss
        val_losses = []
        for batch in val_loader:
            val_bar.update()
            with torch.no_grad():
                x, y = batch
                y = y.long()
                x, y = x.to(device), y.to(device)
                
                output = model(x)
                loss = loss_module(output, y)
                val_losses.append(loss.item())

                del x
                del y
                gc.collect()
        
        val_bar.refresh()
        train_bar.reset()
        val_bar.reset()
        print("Epoch: " + str(epoch + 1) + " Train Loss: " + str(torch.tensor(train_losses).mean()) + " Validation Loss: " + str(torch.tensor(val_losses).mean()))                

In [9]:
model = ResNet(4)
train_model(model, 200)

Epoch: 1 Train Loss: tensor(10.6430) Validation Loss: tensor(2.3646)
Epoch: 2 Train Loss: tensor(1.5907) Validation Loss: tensor(2.7268)
Epoch: 3 Train Loss: tensor(1.5992) Validation Loss: tensor(2.2175)
Epoch: 4 Train Loss: tensor(1.4435) Validation Loss: tensor(1.2679)
Epoch: 5 Train Loss: tensor(1.3608) Validation Loss: tensor(1.2613)
Epoch: 6 Train Loss: tensor(1.2839) Validation Loss: tensor(1.2787)
Epoch: 7 Train Loss: tensor(1.3879) Validation Loss: tensor(1.5695)
Epoch: 8 Train Loss: tensor(1.2648) Validation Loss: tensor(1.2554)
Epoch: 9 Train Loss: tensor(1.2778) Validation Loss: tensor(1.8365)
Epoch: 10 Train Loss: tensor(1.3634) Validation Loss: tensor(1.5861)
Epoch: 11 Train Loss: tensor(1.2981) Validation Loss: tensor(1.3736)
Epoch: 12 Train Loss: tensor(1.3044) Validation Loss: tensor(1.6469)
Epoch: 13 Train Loss: tensor(1.2511) Validation Loss: tensor(1.2539)
Epoch: 14 Train Loss: tensor(1.2254) Validation Loss: tensor(1.5061)
Epoch: 15 Train Loss: tensor(1.2251) Valid

Epoch: 120 Train Loss: tensor(0.0398) Validation Loss: tensor(8.8621)
Epoch: 121 Train Loss: tensor(0.0409) Validation Loss: tensor(8.0848)
Epoch: 122 Train Loss: tensor(0.0313) Validation Loss: tensor(7.4432)
Epoch: 123 Train Loss: tensor(0.0280) Validation Loss: tensor(7.6511)
Epoch: 124 Train Loss: tensor(0.0246) Validation Loss: tensor(7.8841)
Epoch: 125 Train Loss: tensor(0.0158) Validation Loss: tensor(7.4763)
Epoch: 126 Train Loss: tensor(0.0242) Validation Loss: tensor(7.5216)
Epoch: 127 Train Loss: tensor(0.0400) Validation Loss: tensor(7.0260)
Epoch: 128 Train Loss: tensor(0.0528) Validation Loss: tensor(6.7457)
Epoch: 129 Train Loss: tensor(0.0387) Validation Loss: tensor(6.0136)
Epoch: 130 Train Loss: tensor(0.0340) Validation Loss: tensor(7.5248)
Epoch: 131 Train Loss: tensor(0.0403) Validation Loss: tensor(8.0549)
Epoch: 132 Train Loss: tensor(0.0460) Validation Loss: tensor(7.4406)
Epoch: 133 Train Loss: tensor(0.0399) Validation Loss: tensor(7.1095)
Epoch: 134 Train Los

Exception ignored in: <function _releaseLock at 0x7f9b0ab8a670>
Traceback (most recent call last):
  File "/home/kamesh/anaconda3/lib/python3.8/logging/__init__.py", line 223, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


KeyboardInterrupt: 

KeyboardInterrupt: 